In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import iris
plt.rcParams['font.sans-serif']=['Arial']

#### Extract a high-resolution dataset (i.e. 0.5°×0.5°)

In [2]:
Y = 5
lon = 720
lat = 360

In [3]:
res_lon = 360/lon
res_lat = 180/lat
sample_points = [('longitude', np.linspace(0+res_lon/2, 360-res_lon/2, lon)),
                 ('latitude',  np.linspace(-90+res_lat/2, 90-res_lat/2, lat))]

In [4]:
sample_points

[('longitude',
  array([2.5000e-01, 7.5000e-01, 1.2500e+00, 1.7500e+00, 2.2500e+00,
         2.7500e+00, 3.2500e+00, 3.7500e+00, 4.2500e+00, 4.7500e+00,
         5.2500e+00, 5.7500e+00, 6.2500e+00, 6.7500e+00, 7.2500e+00,
         7.7500e+00, 8.2500e+00, 8.7500e+00, 9.2500e+00, 9.7500e+00,
         1.0250e+01, 1.0750e+01, 1.1250e+01, 1.1750e+01, 1.2250e+01,
         1.2750e+01, 1.3250e+01, 1.3750e+01, 1.4250e+01, 1.4750e+01,
         1.5250e+01, 1.5750e+01, 1.6250e+01, 1.6750e+01, 1.7250e+01,
         1.7750e+01, 1.8250e+01, 1.8750e+01, 1.9250e+01, 1.9750e+01,
         2.0250e+01, 2.0750e+01, 2.1250e+01, 2.1750e+01, 2.2250e+01,
         2.2750e+01, 2.3250e+01, 2.3750e+01, 2.4250e+01, 2.4750e+01,
         2.5250e+01, 2.5750e+01, 2.6250e+01, 2.6750e+01, 2.7250e+01,
         2.7750e+01, 2.8250e+01, 2.8750e+01, 2.9250e+01, 2.9750e+01,
         3.0250e+01, 3.0750e+01, 3.1250e+01, 3.1750e+01, 3.2250e+01,
         3.2750e+01, 3.3250e+01, 3.3750e+01, 3.4250e+01, 3.4750e+01,
         3.5250e+01

In [5]:
t = Y*12 ## Y years correspond to 12Y months 

lon_range = np.linspace(0+res_lon/2, 360-res_lon/2, lon)
lat_range = np.linspace(-90+res_lat/2, 90-res_lat/2, lat) 
t_range   = np.linspace(1, t, t) 
lat_mesh, t_mesh, lon_mesh = np.meshgrid(lat_range, t_range, lon_range) 

t_grid   = t_mesh.reshape(t*lon*lat, 1)
lon_grid = lon_mesh.reshape(t*lon*lat, 1)
lat_grid = lat_mesh.reshape(t*lon*lat, 1)

dataset = pd.DataFrame(lon_grid, columns = {'longitude'})
dataset['latitude'] = lat_grid
dataset['t'] = t_grid
dataset['loc'] = dataset['longitude']*1000+dataset['latitude']
dataset

,longitude,latitude,t,loc
0,0.25,-89.75,1.0,160.25
1,0.75,-89.75,1.0,660.25
2,1.25,-89.75,1.0,1160.25
3,1.75,-89.75,1.0,1660.25
4,2.25,-89.75,1.0,2160.25
...,...,...,...,...
15551995,357.75,89.75,60.0,357839.75
15551996,358.25,89.75,60.0,358339.75
15551997,358.75,89.75,60.0,358839.75
15551998,359.25,89.75,60.0,359339.75


In [6]:
data = iris.load_cube('/Volumes/Database/CMIP6/MOHC/historical/o3_AERmon_UKESM1-0-LL_historical_r1i1p1f2_gn_200001-201412.nc', 'o3')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
o3 = values.reshape(t*lon*lat,1)
dataset['o3'] = o3

In [7]:
data = iris.load_cube('/Volumes/Database/CMIP6/MOHC/historical/o3prod_AERmon_UKESM1-0-LL_historical_r1i1p1f2_gn_200001-201412.nc', 'o3prod')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
o3prod = values.reshape(t*lon*lat,1)
dataset['o3prod'] = o3prod

In [8]:
data = iris.load_cube('/Volumes/Database/CMIP6/MOHC/historical/o3loss_AERmon_UKESM1-0-LL_historical_r1i1p1f2_gn_200001-201412.nc', 'o3loss')
time_seq, layer, latitude, longitude = data.shape
data = data[time_seq-t:time_seq,0,:,:]
data = data.interpolate(sample_points, iris.analysis.Linear())
values = np.array(data.data)
o3loss = values.reshape(t*lon*lat,1)
dataset['o3loss'] = o3loss

In [9]:
dataset

,longitude,latitude,t,loc,o3,o3prod,o3loss
0,0.25,-89.75,1.0,160.25,1.635586e-08,8.874727e-14,9.524117e-14
1,0.75,-89.75,1.0,660.25,1.635585e-08,8.873214e-14,9.523655e-14
2,1.25,-89.75,1.0,1160.25,1.635599e-08,8.873399e-14,9.524327e-14
3,1.75,-89.75,1.0,1660.25,1.635621e-08,8.874603e-14,9.525680e-14
4,2.25,-89.75,1.0,2160.25,1.635642e-08,8.875808e-14,9.527032e-14
...,...,...,...,...,...,...,...
15551995,357.75,89.75,60.0,357839.75,1.621541e-08,3.649463e-16,1.918938e-16
15551996,358.25,89.75,60.0,358339.75,1.621581e-08,3.846478e-16,1.967400e-16
15551997,358.75,89.75,60.0,358839.75,1.621621e-08,4.043494e-16,2.015861e-16
15551998,359.25,89.75,60.0,359339.75,1.621423e-08,4.359465e-16,2.069088e-16


In [10]:
dataset_pt1 = dataset[dataset['longitude'] < 180]
dataset_pt2 = dataset[dataset['longitude'] >= 180]
dataset_pt2['longitude'] = dataset_pt2['longitude']-360
dataset_adj = dataset_pt1.append(dataset_pt2)
dataset_adj

,longitude,latitude,t,loc,o3,o3prod,o3loss
0,0.25,-89.75,1.0,160.25,1.635586e-08,8.874727e-14,9.524117e-14
1,0.75,-89.75,1.0,660.25,1.635585e-08,8.873214e-14,9.523655e-14
2,1.25,-89.75,1.0,1160.25,1.635599e-08,8.873399e-14,9.524327e-14
3,1.75,-89.75,1.0,1660.25,1.635621e-08,8.874603e-14,9.525680e-14
4,2.25,-89.75,1.0,2160.25,1.635642e-08,8.875808e-14,9.527032e-14
...,...,...,...,...,...,...,...
15551995,-2.25,89.75,60.0,357839.75,1.621541e-08,3.649463e-16,1.918938e-16
15551996,-1.75,89.75,60.0,358339.75,1.621581e-08,3.846478e-16,1.967400e-16
15551997,-1.25,89.75,60.0,358839.75,1.621621e-08,4.043494e-16,2.015861e-16
15551998,-0.75,89.75,60.0,359339.75,1.621423e-08,4.359465e-16,2.069088e-16


In [17]:
dataset['o3prod_mole'] = dataset['o3prod']*6.022*10**17
dataset['o3loss_mole'] = dataset['o3loss']*6.022*10**17

dataset['o3net'] = dataset['o3prod_mole'] - dataset['o3loss_mole']
dataset['o3_ppb'] = dataset['o3']*10**9

In [18]:
dataset

,longitude,latitude,t,loc,o3,o3prod,o3loss,o3prod_mole,o3loss_mole,o3net,o3_ppb
0,0.25,-89.75,1.0,160.25,1.635586e-08,8.874727e-14,9.524117e-14,53443.605994,57354.232574,-3910.626580,16.355860
1,0.75,-89.75,1.0,660.25,1.635585e-08,8.873214e-14,9.523655e-14,53434.492299,57351.450410,-3916.958111,16.355854
2,1.25,-89.75,1.0,1160.25,1.635599e-08,8.873399e-14,9.524327e-14,53435.606369,57355.497194,-3919.890825,16.355987
3,1.75,-89.75,1.0,1660.25,1.635621e-08,8.874603e-14,9.525680e-14,53442.861675,57363.644960,-3920.783285,16.356205
4,2.25,-89.75,1.0,2160.25,1.635642e-08,8.875808e-14,9.527032e-14,53450.112765,57371.786704,-3921.673939,16.356424
...,...,...,...,...,...,...,...,...,...,...,...
15551995,-2.25,89.75,60.0,357839.75,1.621541e-08,3.649463e-16,1.918938e-16,219.770674,115.558434,104.212240,16.215413
15551996,-1.75,89.75,60.0,358339.75,1.621581e-08,3.846478e-16,1.967400e-16,231.634923,118.476804,113.158119,16.215811
15551997,-1.25,89.75,60.0,358839.75,1.621621e-08,4.043494e-16,2.015861e-16,243.499185,121.395174,122.104011,16.216209
15551998,-0.75,89.75,60.0,359339.75,1.621423e-08,4.359465e-16,2.069088e-16,262.526964,124.600485,137.926479,16.214234


In [22]:
o3net_map = dataset.groupby(['loc']).mean()
o3net_map

,longitude,latitude,t,o3,o3prod,o3loss,o3prod_mole,o3loss_mole,o3net,o3_ppb
loc,,,,,,,,,,
160.25,0.25,-89.75,30.5,1.748498e-08,2.463897e-14,3.095005e-14,14837.586833,18638.121870,-3800.535037,17.484984
160.75,0.25,-89.25,30.5,1.746087e-08,2.490451e-14,3.136068e-14,14997.493136,18885.401326,-3887.908190,17.460865
161.25,0.25,-88.75,30.5,1.743675e-08,2.517004e-14,3.177131e-14,15157.399168,19132.680721,-3975.281553,17.436747
161.75,0.25,-88.25,30.5,1.741263e-08,2.543558e-14,3.218193e-14,15317.305501,19379.960266,-4062.654765,17.412629
162.25,0.25,-87.75,30.5,1.748548e-08,2.591031e-14,3.299281e-14,15603.186111,19868.271844,-4265.085732,17.485477
...,...,...,...,...,...,...,...,...,...,...
359837.75,-0.25,87.75,30.5,2.010986e-08,1.819803e-13,1.321836e-13,109588.534886,79600.979177,29987.555709,20.109860
359838.25,-0.25,88.25,30.5,2.003464e-08,1.973328e-13,1.318272e-13,118833.823628,79386.355388,39447.468239,20.034637
359838.75,-0.25,88.75,30.5,1.999603e-08,2.121464e-13,1.319541e-13,127754.579954,79462.745667,48291.834287,19.996029


In [23]:
o3net_map.to_csv('/Users/csuen/Documents/GitHub/ozone-radical/Fig. 3/Fig. 3a net ozone chemical production rate.csv', index=False)

#### Statistics of ozone chemical budget over the terrestrial land surfaces

In [ ]:
dataset = pd.read_csv('/Users/csuen/radicals/UKESM1-0-LL_2010_2014.csv')

land = pd.read_csv('/Users/csuen/Documents/GitHub/ozone-budget/global_grid_2x2_label_continent.csv')
dataset_land = dataset.merge(land, how='left', left_on=['longitude','latitude'], right_on=['longitude','latitude'])
dataset_land = dataset_land[dataset_land['land_global']==1]

In [26]:
dataset_land['o3net_mole_stacked'] = dataset_land['o3prod_mole_stacked'] - dataset_land['o3loss_mole_stacked']

In [ ]:
map = dataset_land.groupby(['loc']).mean()

In [40]:
map['o3net_mole_stacked'].mean()

4099436.809681778

In [41]:
map['o3net_mole_stacked'].std()

4731150.365645697

In [42]:
map['o3net_mole_stacked'].median()

2197233.7518327264

In [43]:
np.nanpercentile(map['o3net_mole_stacked'],25)

539119.8905382816

In [44]:
np.nanpercentile(map['o3net_mole_stacked'],75)

6192972.892289851

In [45]:
map['o3net_mole_stacked'].max()

30702314.33425014

In [46]:
map['o3net_mole_stacked'].min()

-887334.7642193774

In [47]:
np.nanpercentile(map['o3net_mole_stacked'],5)

-21390.4502841429

In [48]:
np.nanpercentile(map['o3net_mole_stacked'],95)

14270987.545431089